In [122]:
from textblob import TextBlob
import numpy as np
import pandas as pd
import nltk
pd.options.display.max_columns = None
from sklearn import linear_model
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn import set_config
from sklearn.base import clone
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from feature_engine.preprocessing import MatchVariables
from feature_engine.encoding import RareLabelEncoder, OrdinalEncoder, OneHotEncoder
from feature_engine.imputation import (
    AddMissingIndicator, MeanMedianImputer, CategoricalImputer, EndTailImputer
)
from feature_engine.transformation import PowerTransformer
from feature_engine.creation import RelativeFeatures
from feature_engine.selection import DropFeatures
plt.style.use('ggplot')
train = pd.read_csv('/Users/manu/Analisis Predictivo/Final/train.csv')
val = pd.read_csv('/Users/manu/Analisis Predictivo/Final/val.csv')
valpred = pd.read_csv('/Users/manu/Analisis Predictivo/Final/valpred.csv')


In [123]:
val.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28666 entries, 0 to 28665
Data columns (total 49 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Unnamed: 0                    28666 non-null  int64  
 1   student.id                    28666 non-null  float64
 2   generation                    28666 non-null  float64
 3   educational.model             28666 non-null  float64
 4   level                         28666 non-null  float64
 5   gender                        28666 non-null  float64
 6   age                           28666 non-null  float64
 7   max.degree.parents            28666 non-null  float64
 8   father.education.complete     28666 non-null  float64
 9   father.education.summary      28666 non-null  float64
 10  mother.education.complete     28666 non-null  float64
 11  mother.education.summary      28666 non-null  float64
 12  parents.exatec                28666 non-null  float64
 13  f

In [124]:
valpred.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28666 entries, 0 to 28665
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   retention  28666 non-null  float64
dtypes: float64(1)
memory usage: 224.1 KB


In [125]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114660 entries, 0 to 114659
Data columns (total 50 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   Unnamed: 0                    114660 non-null  int64  
 1   student.id                    114660 non-null  float64
 2   generation                    114660 non-null  float64
 3   educational.model             114660 non-null  float64
 4   level                         114660 non-null  float64
 5   gender                        114660 non-null  float64
 6   age                           114660 non-null  float64
 7   max.degree.parents            114660 non-null  float64
 8   father.education.complete     114660 non-null  float64
 9   father.education.summary      114660 non-null  float64
 10  mother.education.complete     114660 non-null  float64
 11  mother.education.summary      114660 non-null  float64
 12  parents.exatec                114660 non-nul

## Regresión Logistica

In [126]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Dividir el conjunto de datos en variables predictoras (X) y variable objetivo (y)
X = train.drop('retention', axis=1)
y = train['retention']

# Crear el modelo de regresión logística
model = LogisticRegression(max_iter = 1000)

# Realizar la validación cruzada
cv_scores = cross_val_score(model, X, y, cv=5)

# Imprimir los puntajes de la validación cruzada
print("Puntajes de la validación cruzada:", cv_scores)
print("Puntaje promedio:", cv_scores.mean())

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=141102)

# Entrenar el modelo con los datos de entrenamiento
model.fit(X_train, y_train)

# Predecir la variable objetivo para los datos de prueba
y_pred = model.predict(X_test)

# Calcular la precisión del modelo
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Calcular el F1-Score
f1 = f1_score(y_test, y_pred)
print("F1-Score:", f1)

# Calcular la precisión
precision = precision_score(y_test, y_pred)
print("Precision:", precision)

# Calcular la sensibilidad (Recall)
recall = recall_score(y_test, y_pred)
print("Recall:", recall)

# Calcular el área bajo la curva ROC (AUC-ROC)
probabilities = model.predict_proba(X_test)[:, 1]
auc_roc = roc_auc_score(y_test, probabilities)
print("AUC-ROC:", auc_roc)


Puntajes de la validación cruzada: [0.91889063 0.91889063 0.91884703 0.91954474 0.91937031]
Puntaje promedio: 0.9191086691086691
Accuracy: 0.9176260247688819
F1-Score: 0.9570437748720864
Precision: 0.9176260247688819
Recall: 1.0
AUC-ROC: 0.6074329839676136


### Resultados en base valpred

In [127]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Dividir la base de datos val en variables predictoras (X_val) y variable objetivo (y_val)
X_val = val

# Utilizar el modelo entrenado para predecir los valores de 'retention' en la base de datos val
y_pred = model.predict(X_val)

# Crear un DataFrame con las predicciones
y_pred_df = pd.DataFrame(y_pred, columns=['retention'])

# Concatenar las predicciones con la columna 'id' de la base 'val'
submission = pd.concat([val['Unnamed: 0'], y_pred_df], axis=1)

# Imprimir el resultado
submission['retention'].value_counts()

1.0    28666
Name: retention, dtype: int64

In [129]:
# Calcular las métricas de desempeño utilizando las predicciones y los valores reales
accuracy = accuracy_score(submission['retention'], valpred['retention'])
precision = precision_score(submission['retention'], valpred['retention'])
recall = recall_score(submission['retention'], valpred['retention'])
f1 = f1_score(submission['retention'], valpred['retention'])
#auc_roc = roc_auc_score(submission['retention'], valpred['retention'])

# Imprimir los resultados
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1)
#print("AUC-ROC:", auc_roc)

Accuracy: 0.9185097327844833
Precision: 1.0
Recall: 0.9185097327844833
F1-Score: 0.9575241835769874


In [109]:
submission
#submission = submission.rename(columns={'Unnamed: 0': 'id'})
#submission.to_csv('submission.csv', index=False)

,id,retention
0,4287,1.0
1,131864,1.0
2,107197,1.0
3,80915,1.0
4,116087,0.0
...,...,...
28661,114622,1.0
28662,59203,1.0
28663,15669,1.0
28664,5310,1.0


## Arboles de Clasficación

In [130]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from skopt import BayesSearchCV

X = train.drop('retention', axis=1)
y = train['retention']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=141102)

# Definir los rangos de los hiperparámetros a ajustar
param_space = {
    'n_estimators': (100, 1000),
    'max_depth': (1, 10),
    'min_samples_split': (2, 10),
    'min_samples_leaf': (1, 5),
}

# Crear el modelo de Bosque Aleatorio (Random Forest)
model = RandomForestClassifier()

# Realizar la búsqueda bayesiana de hiperparámetros
bayes_search = BayesSearchCV(
    model,
    param_space,
    n_iter=50,
    cv=5,
    scoring='accuracy',
    n_jobs=-1
)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=141102)

# Ajustar el modelo con los datos de entrenamiento
bayes_search.fit(X_train, y_train)

# Obtener el mejor modelo con los hiperparámetros ajustados
best_model = bayes_search.best_estimator_

# Predecir la variable objetivo para los datos de prueba
y_pred = best_model.predict(X_test)

# Calcular la precisión del modelo
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Calcular el F1-Score
f1 = f1_score(y_test, y_pred)
print("F1-Score:", f1)

# Calcular la precisión
precision = precision_score(y_test, y_pred)
print("Precision:", precision)

# Calcular la sensibilidad (Recall)
recall = recall_score(y_test, y_pred)
print("Recall:", recall)

# Calcular el área bajo la curva ROC (AUC-ROC)
probabilities = best_model.predict_proba(X_test)[:, 1]
auc_roc = roc_auc_score(y_test, probabilities)
print("AUC-ROC:", auc_roc)


Accuracy: 0.9229461015175301
F1-Score: 0.9596160438806994
Precision: 0.9243571680169074
Recall: 0.9976714346813668
AUC-ROC: 0.7364167882613601


In [131]:
# Imprimir los mejores hiperparámetros
print("Mejores hiperparámetros encontrados:")
print(best_model.get_params())

Mejores hiperparámetros encontrados:
{'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': 10, 'max_features': 'auto', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 10, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 166, 'n_jobs': None, 'oob_score': False, 'random_state': None, 'verbose': 0, 'warm_start': False}


In [132]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Dividir la base de datos val en variables predictoras (X_val) y variable objetivo (y_val)
X_val = val

# Utilizar el modelo entrenado para predecir los valores de 'retention' en la base de datos val
y_pred = best_model.predict(X_val)

# Crear un DataFrame con las predicciones
y_pred_df = pd.DataFrame(y_pred, columns=['retention'])

# Concatenar las predicciones con la columna 'id' de la base 'val'
submission = pd.concat([val['Unnamed: 0'], y_pred_df], axis=1)

# Imprimir el resultado
submission['retention'].value_counts()

1.0    28389
0.0      277
Name: retention, dtype: int64

In [133]:
# Calcular las métricas de desempeño utilizando las predicciones y los valores reales
accuracy = accuracy_score(submission['retention'], valpred['retention'])
precision = precision_score(submission['retention'], valpred['retention'])
recall = recall_score(submission['retention'], valpred['retention'])
f1 = f1_score(submission['retention'], valpred['retention'])
auc_roc = roc_auc_score(submission['retention'], valpred['retention'])

# Imprimir los resultados
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1)
print("AUC-ROC:", auc_roc)

Accuracy: 0.9248936021767948
Precision: 0.9982149639194835
Recall: 0.9258163373137482
F1-Score: 0.9606535207149253
AUC-ROC: 0.8780706235305203


In [134]:
submission
#submission = submission.rename(columns={'Unnamed: 0': 'id'})
#submission.to_csv('submission.csv', index=False)

,Unnamed: 0,retention
0,4287,1.0
1,131864,1.0
2,107197,1.0
3,80915,1.0
4,116087,1.0
...,...,...
28661,114622,1.0
28662,59203,1.0
28663,15669,1.0
28664,5310,1.0


## XGBoost

In [136]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from skopt import BayesSearchCV

# Dividir el conjunto de datos en variables predictoras (X) y variable objetivo (y)
X = train.drop('retention', axis=1)
y = train['retention']

# Definir los rangos de los hiperparámetros a ajustar
param_space = {
    'n_estimators': (100, 1000),
    'max_depth': (1, 10),
    'learning_rate': (0.01, 0.1),
    'subsample': (0.5, 1.0),
    'colsample_bytree': (0.5, 1.0),
}

# Crear el modelo XGBoost
model = xgb.XGBClassifier()

# Realizar la búsqueda bayesiana de hiperparámetros
bayes_search = BayesSearchCV(
    model,
    param_space,
    n_iter=50,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    verbose=2
)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=141102)

# Ajustar el modelo con los datos de entrenamiento
bayes_search.fit(X_train, y_train)

# Obtener el mejor modelo con los hiperparámetros ajustados
best_model = bayes_search.best_estimator_

# Imprimir los mejores hiperparámetros
print("Mejores hiperparámetros encontrados:")
print(bayes_search.best_params_)

# Predecir la variable objetivo para los datos de prueba
y_pred = best_model.predict(X_test)

# Calcular la precisión del modelo
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Calcular el F1-Score
f1 = f1_score(y_test, y_pred)
print("F1-Score:", f1)

# Calcular la precisión
precision = precision_score(y_test, y_pred)
print("Precision:", precision)

# Calcular la sensibilidad (Recall)
recall = recall_score(y_test, y_pred)
print("Recall:", recall)

# Calcular el área bajo la curva ROC (AUC-ROC)
probabilities = best_model.predict_proba(X_test)[:, 1]
auc_roc = roc_auc_score(y_test, probabilities)
print("AUC-ROC:", auc_roc)


Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END colsample_bytree=0.5858201209966899, learning_rate=0.06586909912038931, max_depth=1, n_estimators=409, subsample=0.5105513538762654; total time=  27.1s
[CV] END colsample_bytree=0.8245173154565024, learning_rate=0.05340473192140112, max_depth=1, n_estimators=681, subsample=0.935469631266669; total time=  54.7s
[CV] END colsample_bytree=0.8245173154565024, learning_rate=0.05340473192140112, max_depth=1, n_estimators=681, subsample=0.935469631266669; total time=  54.1s
[CV] END colsample_bytree=0.9505019933314707, learning_rate=0.08420137773086178, max_depth=1, n_estimators=700, subsample=0.9216897162951836; total time= 1.0min
[CV] END colsample_bytree=0.7333243859146015, learning_rate=0.0

[CV] END colsample_bytree=0.9665504863760621, learning_rate=0.09980036538330848, max_depth=3, n_estimators=972, subsample=0.8683734647474215; total time= 4.0min
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END colsample_bytree=0.5858201209966899, learning_rate=0.06586909912038931, max_depth=1, n_estimators=409, subsample=0.5105513538762654; total time=  27.1s
[CV] END colsample_bytree=0.8245173154565024, learning_rate=0.05340473192140112, max_depth=1, n_estimators=681, subsample=0.935469631266669; total time=  54.6s
[CV] END colsample_bytree=0.6503107176233643, learning_rate=0.07475454928151215, max_depth=10, n_estimators=838, subsample=0.950733623272952; total time= 7.6min
[CV] END colsample_bytree=0.796017149814095, learning_rate=0.022300007874398772, max_depth=5, n_estimators=940, subsample

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END colsample_bytree=0.746513618686454, learning_rate=0.033935915420397, max_depth=10, n_estimators=307, subsample=0.624065436711099; total time= 2.0min
[CV] END colsample_bytree=0.9986621970640247, learning_rate=0.011855410888713567, max_depth=9, n_estimators=796, subsample=0.8121142637776284; total time= 5.9min
[CV] END colsample_bytree=0.5258211726499038, learning_rate=0.01483963560336309, max_depth=1, n_estimators=956, subsample=0.5097313267387357; total time=  35.4s
[CV] END colsample_bytree=0.5, learning_rate=0.051969245470268696, max_depth=10, n_estimators=100, subsample=1.0; total time=  28.4s
[CV] END colsample_bytree=0.5, learning_rate=0.03154106882369517, max_depth=7, n_estimators=455, subsample=0.7764194198781027; total time= 1.6min
[CV] END colsample_bytree=0.9867672907163776, learning_rate=0.0

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END colsample_bytree=0.5, learning_rate=0.1, max_depth=1, n_estimators=1000, subsample=0.612687402710845; total time=  38.3s
[CV] END colsample_bytree=0.5094867434135308, learning_rate=0.09975356964793877, max_depth=10, n_estimators=967, subsample=0.8587486215055194; total time= 5.2min
[CV] END colsample_bytree=0.5, learning_rate=0.0612334114526514, max_depth=6, n_estimators=174, subsample=1.0; total time=  35.6s
[CV] END colsample_bytree=0.567847516111302, learning_rate=0.06119117349493012, max_depth=2, n_estimators=999, subsample=0.5387769996793631; total time= 1.3min
[CV] END colsample_bytree=0.9907387969736468, learning_rate=0.09869809843404856, max_depth=10, n_estimators=162, subsample=0.9457094402969182; total time= 1.7min
[CV] END colsample_bytree=0.5, learning_rate=0.01, max_depth=7, n_estimators=1000, subsample=1.0; total time= 3.8min
[CV] END colsample_bytree=0.5, learning_rate=0.01, max_depth=10, n_estimators=70

In [137]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Dividir la base de datos val en variables predictoras (X_val) y variable objetivo (y_val)
X_val = val

# Utilizar el modelo entrenado para predecir los valores de 'retention' en la base de datos val
y_pred = best_model.predict(X_val)

# Crear un DataFrame con las predicciones
y_pred_df = pd.DataFrame(y_pred, columns=['retention'])

# Concatenar las predicciones con la columna 'id' de la base 'val'
submission = pd.concat([val['Unnamed: 0'], y_pred_df], axis=1)

# Imprimir el resultado
submission['retention'].value_counts()

1    28303
0      363
Name: retention, dtype: int64

In [138]:
# Calcular las métricas de desempeño utilizando las predicciones y los valores reales
accuracy = accuracy_score(submission['retention'], valpred['retention'])
precision = precision_score(submission['retention'], valpred['retention'])
recall = recall_score(submission['retention'], valpred['retention'])
f1 = f1_score(submission['retention'], valpred['retention'])
auc_roc = roc_auc_score(submission['retention'], valpred['retention'])

# Imprimir los resultados
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1)
print("AUC-ROC:", auc_roc)

Accuracy: 0.9254517546919696
Precision: 0.9968856817318648
Recall: 0.9273928558810021
F1-Score: 0.9608844471290245
AUC-ROC: 0.8507487695382971
[CV] END colsample_bytree=0.9907387969736468, learning_rate=0.09869809843404856, max_depth=10, n_estimators=162, subsample=0.9457094402969182; total time= 1.7min
[CV] END colsample_bytree=0.5, learning_rate=0.01, max_depth=10, n_estimators=709, subsample=1.0; total time= 4.4min
[CV] END colsample_bytree=0.9873266748859229, learning_rate=0.010437705852811905, max_depth=2, n_estimators=821, subsample=0.9842081525721144; total time= 1.6min
[CV] END colsample_bytree=0.5, learning_rate=0.05491671418476505, max_depth=6, n_estimators=100, subsample=0.5; total time=  19.7s
[CV] END colsample_bytree=0.5562805677376491, learning_rate=0.011019280339427484, max_depth=5, n_estimators=113, subsample=0.9932065255733373; total time=  20.5s
[CV] END colsample_bytree=0.9274110524199934, learning_rate=0.0903628930447217, max_depth=6, n_estimators=232, subsample=0.

In [108]:
submission = submission.rename(columns={'Unnamed: 0': 'id'})
submission.to_csv('submission.csv', index=False)


In [107]:
submission


,Unnamed: 0,retention
0,4287,1.0
1,131864,1.0
2,107197,1.0
3,80915,1.0
4,116087,0.0
...,...,...
28661,114622,1.0
28662,59203,1.0
28663,15669,1.0
28664,5310,1.0
